# Creating a real tomography dataset
Using the true (clean) dataset, crate a dataset matched with the simulated one.

In [ ]:
from utils import *
from common import *

In [2]:
N_MAX = 69352 # train + validation
# N = int(N_MAX*10/11) # train
N = 50_000 # train
# N = 10_000 # train

In [ ]:
# load data
data = np.load('data/data_clean.npz')
print(data.files)
for k in data.files:
    print(k, data[k].shape) 
sxr = data['sxr']
emiss = data['emiss']
#cehck all the values in x_emiss and y_emiss are the same
xemiss0, yemiss0 = data['x_emiss'][0], data['y_emiss'][0]
for xemiss, yemiss in zip(data['x_emiss'], data['y_emiss']):
    assert np.allclose(xemiss, xemiss0)
    assert np.allclose(yemiss, yemiss0)
rr, zz = data['x_emiss'][0], data['y_emiss'][0]
del data

In [4]:
# # emiss info
# max_emiss, min_emiss, mean_emiss, std_emiss = np.max(emiss), np.min(emiss), np.mean(emiss), np.std(emiss)
# max_sxr, min_sxr, mean_sxr, std_sxr = np.max(sxr), np.min(sxr), np.mean(sxr), np.std(sxr)
# print(f'max_emiss={max_emiss}, min_emiss={min_emiss}, mean_emiss={mean_emiss}, std_emiss={std_emiss}')

# # #plot the historgram of emiss
# # emiss_flat = emiss.flatten()
# # plt.figure(figsize=(10,3))
# # plt.subplot(121)
# # plt.hist(emiss_flat, bins=100)  
# # plt.subplot(122)
# # plt.hist(emiss_flat, bins=100)  #looking at this 3000 is a good value
# # plt.ylim(0, 5000)
# # plt.title('emiss histogram')
# # plt.show() 

# # plot the histograms of max(emiss)
# emiss_flat = emiss.reshape(-1, emiss.shape[1]*emiss.shape[2])
# max_emiss = np.max(emiss_flat, axis=1)
# print(max_emiss.shape)

# plt.figure(figsize=(15,3))
# plt.subplot(141)
# plt.hist(max_emiss, bins=100)
# plt.xlim(0, 3000)
# plt.subplot(142)
# plt.hist(max_emiss, bins=100)
# plt.ylim(0, 500); plt.xlim(0, 3000)
# plt.title('max emiss histogram')

# # fit to a distribution (lognorm or gamma) lognorm better
# from scipy.stats import lognorm, gamma
# # fit a lognorm distribution
# shape, loc, scale = lognorm.fit(max_emiss) 
# # fit a gamma distribution
# # shape, loc, scale = gamma.fit(max_emiss)

# print(f'distribution: shape={shape}, loc={loc}, scale={scale}')

# x = np.linspace(0, 3000, 100)
# y = lognorm.pdf(x, shape, loc, scale)*np.sum(max_emiss)
# # y = gamma.pdf(x, shape, loc, scale)*np.sum(max_emiss)

# s = lognorm.rvs(shape, loc, scale, size=len(max_emiss))
# # s = gamma.rvs(shape, loc, scale, size=len(max_emiss))

# plt.subplot(143)
# plt.plot(x, y)
# plt.title('fit')
# plt.subplot(144)

# plt.hist(s, bins=100)
# plt.xlim(0, 3000)

# plt.show()



# # # kind of normalization, coherent with the integral nature of the problem # NOW done in the dataset
# # K_NORM = 3000
# # sxr = sxr/K_NORM
# # emiss = emiss/K_NORM

In [5]:
# split sxr into vdi, vdc, vde, hor1
vdi = sxr[:, VDI_INTERVAL[0]:VDI_INTERVAL[1]]
vdc = sxr[:, VDC_INTERVAL[0]:VDC_INTERVAL[1]]
vde = sxr[:, VDE_INTERVAL[0]:VDE_INTERVAL[1]]
hor = sxr[:, HOR1_INTERVAL[0]:HOR1_INTERVAL[1]]

In [ ]:
# get emiss maxes to get the scaling factors
emiss_maxes = np.max(emiss.reshape(-1, emiss.shape[1]*emiss.shape[2]), axis=1)
print(emiss_maxes.shape)
print(emiss_maxes[:10])

In [7]:
# scale
vdi_s = vdi/emiss_maxes[:,None]
vdc_s = vdc/emiss_maxes[:,None]
vde_s = vde/emiss_maxes[:,None]
hor_s = hor/emiss_maxes[:,None]

In [ ]:
# plot distributions
fans = [vdi_s, vdc_s, vde_s, hor_s]

FILTER = -10 #0.01 


# plot histograms
for fan, color, name in zip(fans, ['r', 'g', 'b', 'y'], ['VDI', 'VDC', 'VDE', 'HOR']):
    nrays = fan.shape[1]
    plt.figure(figsize=(15, 1*nrays))
    for i in range(nrays):
        plt.subplot(nrays, 1, i+1)
        r = fan[:,i]
        r = r[r > FILTER]
        μ, σ = np.mean(r), np.std(r)
        # define the bins eqispaced from 0 to 0.8
        bins = np.linspace(-0.05, .7, 100)
        hist = np.histogram(r, bins=bins)[0]
        #histogram
        plt.hist(r, bins=bins, color=color, alpha=0.8)
        plt.plot([μ, μ], [0, np.max(hist)], 'k', lw=3)
        plt.grid(False)
        if i < nrays-1: plt.xticks([]) 
        plt.yticks([])
        plt.gca().spines['top'].set_visible(False)
        plt.gca().spines['left'].set_visible(False)
        plt.gca().spines['right'].set_visible(False)
        # plt.axis('off')
    plt.suptitle(name)
    plt.tight_layout()
    plt.show()
    plt.close()

# same plot but unknown pleasures style
plt.figure(figsize=(15, 10))
for j, (fan, color, name) in enumerate(zip(fans, ['r', 'g', 'b', 'y'], ['VDI', 'VDC', 'VDE', 'HOR'])):
    nrays = fan.shape[1]
    plt.subplot(2, 2, j+1)
    for i in range(nrays):
        r = fan[:,i]
        r = r[r > FILTER]
        # define the bins eqispaced from 0 to 0.8
        bins = np.linspace(-0.05, .7, 100)
        #histogram
        hist = np.histogram(r, bins=bins)[0]/len(r)
        # plot the histogram
        plt.plot(bins[:-1], hist+i/20, color=color, lw=2, alpha=0.99)
        plt.plot([-0.025, .65], [i/20, i/20], f'{color}--', lw=1, alpha=0.99)
        plt.fill_between(bins[:-1], hist+i/20, i/20, color=color, alpha=.5)
        plt.grid(False)
        # plt.axis('off')
        plt.ylim(0, (nrays+1)/20)
        plt.yticks([])
    plt.title(name)
plt.tight_layout()
plt.show()

# plot the mean and std of the distributions
means, stds = [np.zeros(f.shape[1]) for f in fans], [np.zeros(f.shape[1]) for f in fans]



plt.figure(figsize=(15,10))
for j, (fan, color, name) in enumerate(zip(fans, ['r', 'g', 'b', 'y'], ['VDI', 'VDC', 'VDE', 'HOR'])):
    nrays = fan.shape[1]
    plt.subplot(2, 2, j+1)
    for i in range(nrays):
        r = fan[:,i]
        r = r[r > FILTER]
        # get mean and std
        mean, std = np.mean(r), np.std(r)
        means[j][i], stds[j][i] = mean, std
        # error bar
        plt.errorbar(i, mean, yerr=std, c=color, fmt='o')
    plt.plot(means[j], c=color, label=name)
    plt.fill_between(range(len(means[j])), means[j]-stds[j], means[j]+stds[j], color=color, alpha=0.3)
    plt.ylim(0, .7)
    plt.grid(axis='x')
    plt.title(name)
plt.tight_layout()
plt.show()

#concatenate the means and stds (check dataset in utils.py)
means_conc = np.concatenate(means)
stds_conc = np.concatenate(stds)
print(f'means=\n{means}, \nstds=\n{stds}')

# save the stds and means concatenated for the dataset
np.savez(f'data/rfx_sxr_means_stds_{GSIZE}.npz', means=means_conc, stds=stds_conc)

In [ ]:
# convert dataset in SXRDataset loadable format
nt, nv = N, N//10
assert nt + nv <= N_MAX
print(f'nt={nt}, nv={nv}')
idxs = np.random.randint(0, N, nt+nv) # select nt+nv random samples
tidxs = idxs[:nt] # train
vidxs = idxs[nt:] # validation
emiss_t = np.zeros((nt, GSIZE, GSIZE), dtype=np.float32) # low res
emiss_v = np.zeros((nv, GSIZE, GSIZE), dtype=np.float32) # low res
n_vdi = VDI_INTERVAL[1] - VDI_INTERVAL[0]
n_vdc = VDC_INTERVAL[1] - VDC_INTERVAL[0]
n_vde = VDE_INTERVAL[1] - VDE_INTERVAL[0]
n_hor = HOR1_INTERVAL[1] - HOR1_INTERVAL[0]
vdi_t, vdi_v = np.zeros((nt, n_vdi), dtype=np.float32), np.zeros((nv, n_vdi), dtype=np.float32)
vdc_t, vdc_v = np.zeros((nt, n_vdc), dtype=np.float32), np.zeros((nv, n_vdc), dtype=np.float32)
vde_t, vde_v = np.zeros((nt, n_vde), dtype=np.float32), np.zeros((nv, n_vde), dtype=np.float32)
hor_t, hor_v = np.zeros((nt, n_hor), dtype=np.float32), np.zeros((nv, n_hor), dtype=np.float32)
for i, idx in enumerate(tidxs):
    emiss_t[i] = resize2d(emiss[idx], (GSIZE, GSIZE))
    vdi_t[i], vdc_t[i], vde_t[i], hor_t[i] = vdi[idx], vdc[idx], vde[idx], hor[idx] 
for i, idx in enumerate(vidxs):
    emiss_v[i] = resize2d(emiss[idx], (GSIZE, GSIZE))
    vdi_v[i], vdc_v[i], vde_v[i], hor_v[i] = vdi[idx], vdc[idx], vde[idx], hor[idx]
np.savez(f'data/sxr_real_ds_gs{GSIZE}_n{N}.npz', emiss=emiss_t, vdi=vdi_t, vdc=vdc_t, vde=vde_t, hor=hor_t, RR=RR, ZZ=ZZ)
np.savez(f'data/sxr_real_ds_gs{GSIZE}_n{N//10}.npz', emiss=emiss_v, vdi=vdi_v, vdc=vdc_v, vde=vde_v, hor=hor_v, RR=RR, ZZ=ZZ)


In [ ]:
# load the dataset
data = np.load(f'data/sxr_real_ds_gs{GSIZE}_n{N}.npz')
print(data.files)
# extract the data
emiss, vdi, vdc, vde, hor = data['emiss'], data['vdi'], data['vdc'], data['vde'], data['hor']
print(emiss.shape, vdi.shape, vdc.shape, vde.shape, hor.shape)

In [ ]:
# plot the dataset
N_PLOTS = 10
idxs = np.random.randint(0, N//10, N_PLOTS)
for i in idxs:
    plt.figure(figsize=(10,3))
    mind, maxd = np.min(emiss[i]), np.max(emiss[i])
    # distribution
    plt.subplot(121)
    plt.scatter(RR, ZZ, c=emiss[i], s=6*150/GSIZE, marker='s')
    plt.axis('equal')
    plt.xlim(R0, R1), plt.ylim(Z0, Z1)
    plt.colorbar()
    plt.clim(mind, maxd)
    plt.title(f'Distribution {i}')
    plt.grid(False)
    # sxr
    plt.subplot(122)
    plt.plot(vdi[i], 'rs:', label='VDI')
    plt.plot(vdc[i], 'gs:', label='VDC')
    plt.plot(vde[i], 'bs:', label='VDE')
    plt.plot(hor[i], 'ys:', label='HOR')
    plt.grid(True)
    plt.legend()
    plt.title(f'SXR {i}')
    plt.show()
    plt.close()